In [9]:
import pandas as pd
df=pd.read_csv(r"C:\Users\batch1\Downloads\logs_dataset.csv")

In [10]:
df.head()

,@timestamp,_id,ip_address
0,"July 8th 2019, 14:43:03.000",XswJ0msBoTGddM7vxMDB,10.1.1.285
1,"July 8th 2019, 14:43:01.000",dKQJ0msB7mP0GwVzvJjz,10.1.2.389
2,"July 8th 2019, 14:42:59.000",CcwJ0msBoTGddM7vtb8y,10.1.1.415
3,"July 8th 2019, 14:42:57.000",bKQJ0msB7mP0GwVzrZdT,10.1.1.79
4,"July 8th 2019, 14:42:55.000",L6QJ0msB7mP0GwVzpZeI,10.1.1.60


In [11]:
df['@timestamp'] = (
    df['@timestamp']
    .str.replace(r'(\d+)(st|nd|rd|th)', r'\1', regex=True)
)

In [12]:
df['@timestamp'] = pd.to_datetime(
    df['@timestamp'],
    format='%B %d %Y, %H:%M:%S.%f'
)

In [13]:
df.sort_values(['ip_address', '@timestamp'], inplace=True)
df['shift_time'] = df.groupby(['ip_address'])['@timestamp'].shift(1)
df.head()

,@timestamp,_id,ip_address,shift_time
721473,2019-06-09 00:06:09,DBuOOWsB7mP0GwVzhZ9U,10.1.1.1,NaT
720483,2019-06-09 01:28:39,bB7aOWsB7mP0GwVzDY5G,10.1.1.1,2019-06-09 00:06:09
719233,2019-06-09 03:12:49,R0w5OmsBoTGddM7vayZT,10.1.1.1,2019-06-09 01:28:39
719222,2019-06-09 03:13:45,U0w6OmsBoTGddM7vRi8R,10.1.1.1,2019-06-09 03:12:49
718875,2019-06-09 03:42:39,z01UOmsBoTGddM7vuzyC,10.1.1.1,2019-06-09 03:13:45


In [14]:
df['time_diff'] = (df['@timestamp']-df['shift_time']).dt.seconds//60

In [15]:
df['date'] = df['@timestamp'].dt.date

In [19]:
df['weekday'] = df['@timestamp'].dt.weekday

In [20]:
df['hour'] = df['@timestamp'].dt.hour

In [21]:
df['is_weekend'] = ((df['weekday'] == 5) | (df['weekday'] == 6)).astype(int)

In [22]:
df['hour_bucket']=df['hour']//4

In [23]:
ip_addr='ip_address'

In [24]:
ip_counts=df.groupby(ip_addr)['@timestamp'].count().reset_index()

In [25]:
ip_counts.head()

,ip_address,@timestamp
0,10.1.1.1,1446
1,10.1.1.100,2860
2,10.1.1.101,1465
3,10.1.1.106,1408
4,10.1.1.109,1459


In [26]:
ip_counts=ip_counts.rename(columns={'@timestamp':'total_count'})

In [27]:
daily_counts=df.groupby([ip_addr,'date'])['@timestamp'].count().reset_index()

In [28]:
daily_counts=daily_counts.rename(columns={'@timestamp':'daily_counts'})

In [29]:
daily_counts_avg=daily_counts.groupby(ip_addr).daily_counts.median().reset_index()

In [30]:
daily_counts_avg.head(5)

,ip_address,daily_counts
0,10.1.1.1,40.0
1,10.1.1.100,78.0
2,10.1.1.101,40.0
3,10.1.1.106,35.5
4,10.1.1.109,42.5


In [31]:
weekend_counts=df.groupby([ip_addr,'is_weekend'])['@timestamp'].count().reset_index()

In [32]:
weekend_counts

,ip_address,is_weekend,@timestamp
0,10.1.1.1,0,975
1,10.1.1.1,1,471
2,10.1.1.100,0,1960
3,10.1.1.100,1,900
4,10.1.1.101,0,1006
...,...,...,...
767,10.1.2.90,1,871
768,10.1.2.95,0,1973
769,10.1.2.95,1,895
770,10.1.2.99,0,978
